# Paper Example:

We will use the PCE_calculator module to perform the calculations of Section 5 ("Application Exmaple") in the paper:

A. Silva, D. Pizarro, and B. Stojadinovic, “Displacement prediction equations for seismic design of single
friction pendulum base-isolated structures”, In press.

We start by setting up the input hazard curves and functions to calculate the fragility curve and the mean annual frequency of exceeding limit state thresholds.

In [1]:
import numpy as np
# First step: input the hazard curves values for San Jose (California) site type D
# Input Hazard Curves:
## Hazard curve for Sa(T=0.5s) in g
SaT1_0p5 = np.array([0.0033, 0.0050, 0.0075, 0.0112, 0.0169, 0.0253, 0.0379, 0.0569, 0.0853, 0.1280,
            0.1920, 0.2880, 0.4320, 0.6480, 0.9720, 1.4600, 2.1900, 3.2800, 4.9200, 7.3800]) # in g
MAF_SaT1_0p5 = np.array([1.4112e+00, 1.2319e+00, 1.0402e+00, 8.4929e-01, 6.6328e-01, 4.9971e-01, 3.6208e-01,
                         2.5266e-01, 1.7053e-01, 1.1107e-01, 6.9974e-02, 4.2369e-02, 2.4013e-02, 1.2132e-02,
                         5.1558e-03, 1.7363e-03, 4.3700e-04, 7.3826e-05, 6.7674e-06, 2.0466e-07 ])
## Hazard curve for Sa(T=0.75s) in g
SaT1_0p75 = np.array([0.0025, 0.0037, 0.0056, 0.0084, 0.0126, 0.0190, 0.0284, 0.0427, 0.0640, 0.0960, 
             0.1440, 0.2160, 0.3240, 0.4860, 0.7290, 1.0900, 1.6400, 2.4600, 3.6900, 5.5400]) # in g
MAF_SaT1_0p75 = np.array([1.3966e+00, 1.2132e+00, 1.0185e+00, 8.2311e-01, 6.4004e-01, 4.7430e-01, 3.4012e-01,
                          2.3429e-01, 1.5677e-01, 1.0175e-01, 6.4261e-02, 3.9356e-02, 2.2869e-02, 1.2098e-02,
                          5.5390e-03, 2.1068e-03, 6.1393e-04, 1.2896e-04, 1.6406e-05, 9.0019e-07])
## Hazard curve for PGV in cm/s
PGV = np.array([0.2370, 0.3550, 0.5320, 0.7980, 1.1900, 1.8000, 2.6900, 4.0400, 6.0600, 9.0900, 13.6000,
       20.5000, 30.7000, 46.0000, 69.0000, 103.0000, 155.0000, 233.0000, 349.0000, 525.0000]) # in cm/s
MAF_PGV = np.array([1.1289e+00, 9.4068e-01, 7.5691e-01, 5.8529e-01, 4.3678e-01, 3.1096e-01, 2.1694e-01,
                    1.4654e-01, 9.6565e-02, 6.2430e-02, 3.9847e-02, 2.4332e-02, 1.3755e-02, 6.7175e-03,
                    2.6775e-03, 8.4872e-04, 1.9644e-04, 3.1402e-05, 2.9382e-06, 8.5668e-08])

In [2]:
# Second Step: import functions and define the functions needed for the calculations
from PCE_calculator import InputChecker
from PCE_calculator import DataLoader
from PCE_calculator import PCECalculator
from scipy.stats import norm

def interpolate_HCs_SaT(HC1,MAF1,T1,HC2,MAF2,T2,T3):
    """
    Calculate the interpolation between two hazard curves at periods T1 and T2 and return the hazard curve for period T3

    Args:
    - HC1,HC2 = hazard curve values (Sa(T))
    - MAF1,MAF2 = mean annual frequency of exceedance of each hazard curve
    - T1,T2 : period linked to the IM.
    - T3: period of the output hazard curve

    Returns:
    - HC3, MAF3 = hazard curve values (IMs) and mean annual frequency of exceedance of Sa(T3)
    """
    sa_x = np.linspace(min(min(HC1), min(HC2)), max(max(HC1), max(HC2)), 1000)
    MAF1_interp = np.exp(np.interp(np.log(sa_x),np.log(HC1),np.log(MAF1)))
    MAF2_interp = np.exp(np.interp(np.log(sa_x),np.log(HC2),np.log(MAF2)))
    MAF3_interp = np.zeros(1000)
    for i in range(len(MAF1_interp)):
        #MAF3_interp[i] = np.exp(np.log(MAF1_interp) + ((T3 - T1) / (T2 - T1)) * (np.log(MAF2_interp) - np.log(MAF1_interp)))
        MAF3_interp[i] = np.exp(np.interp(T3,[T1, T2],[np.log(MAF1_interp[i]), np.log(MAF2_interp[i])]))
    return sa_x, MAF3_interp


def frag_from_coefs(limit, IMs, c1, c2, beta):
    """
    Calculate the fragility function based on the cloud coefficients c1, c2, and beta.
    The fragility function is the probability of exceeding a "limit" given an "IM".
    The formula of the cloud analysis is: ln(displacement) = c1 + c2 * ln(IM) + beta

    Args:
    - limit: limit threshold.
    - c1, c2, beta = cloud coefficients
    - IMs: Vector of intensity measure values (herein, Sd(T1) and PGV).

    Returns:
    - fragility: Vector of the fragility curve given IMs.
    """
    # Calculate the argument of the CDF
    argument = (np.log(limit) - (c1 + c2 * np.log(IMs))) / beta

    # Calculate the CDF using scipy.stats.norm.cdf
    fragility = 1 - norm.cdf(argument, 0, 1)

    return fragility

def calculate_MAF(IM,HC,frag,limit):
    """
    Calculate the MAF of exceeding a limit state threshold given a hazard curve and a fragility function.

    Args:
    - IM : vector of IM that the fragility and the hazard curve are a function of.
    - HC : vector of hazard curve MAF(IM).
    - frag : fragility curve defined as a function of IM, thus: P(x>threshold | IM)
    - limit: limit threshold.

    Returns:
    - MAF: Mean annual frequency of exceeding the limit
    """
    # Calculate the derivative of the fragility curve:
    derivative_fragility = np.diff(frag) / np.diff(IM)
    derivative_fragility = np.insert(derivative_fragility, 0, 0)
    
    # Calculate the intergral of (HC*dFrag) with trapezium rule
    product = derivative_fragility * HC
    MAF = np.trapz(product,IM)
    
    return MAF

Before starting the calculation, we must ensure that the hazard curves are in the correct unities (meters and meters/sec) and increase their granularity by linearly interpolating them in the log space.

In [3]:
# 3rd step: Hazard curve treatment
## 3.1. Make sure the hazard curves are in the correct unities,
SdT1_0p5 = SaT1_0p5 * 9.81 * (0.5/2/np.pi)**2  # in m
SdT1_0p75 = SaT1_0p75 * 9.81 * (0.75/2/np.pi)**2  # in m
PGV_m = PGV / 100 # in m

## 3.2 Interpolate the hazard curves for different periods (if needed)
SaT1_0p6, MAF_SaT1_0p6 = interpolate_HCs_SaT(SaT1_0p5,MAF_SaT1_0p5,0.5,SaT1_0p75,MAF_SaT1_0p75,0.75,0.6)
SdT1_0p6 = SaT1_0p6 * 9.81 * (0.6/2/np.pi)**2  # in m

## 3.3. Make the hazard curves more granular by linear interpolating in the log space
SdT1_0p5_interp = np.linspace(min(SdT1_0p5), max(SdT1_0p5), 1000)
MAF_SdT1_0p5_interp = np.exp(np.interp(np.log(SdT1_0p5_interp),np.log(SdT1_0p5),np.log(MAF_SaT1_0p5)))

SdT1_0p75_interp = np.linspace(min(SdT1_0p75), max(SdT1_0p75), 1000)
MAF_SdT1_0p75_interp = np.exp(np.interp(np.log(SdT1_0p75_interp),np.log(SdT1_0p75),np.log(MAF_SaT1_0p75)))

PGV_interp = np.linspace(min(PGV_m), max(PGV_m), 1000)
MAF_PGV_interp = np.exp(np.interp(np.log(PGV_interp),np.log(PGV_m),np.log(MAF_PGV)))


Now we do the calculations for the first line of Table 3 in the paper:

In [4]:
# 4th step: calculate MAF of limit states (values of Table 3 in the paper)
# Input for the PCE metamodeL:
T1 = 0.5 # possible from 0.1 to 1, in seconds
mratio = 0.55 # possible from 0.3 to 0.9
muf = 0.05 # possible from 0.03 to 0.18
Tb = 4.5 # possible from 3 to 6, in seconds

# Threshold limit for fragility function:
frag_lim_D1 = 0.084 # in meters
frag_lim_u0 = 0.40 # in meters

# Calculate PCE
c1D1, c2D1, betaD1 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'D1','GM')
c1u0, c2u0, betau0 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'u0','GM')

# Calculate fragility
fragilityD1 = frag_from_coefs(frag_lim_D1, SdT1_0p5_interp, c1D1, c2D1, betaD1)
fragilityu0 = frag_from_coefs(frag_lim_u0, PGV_interp, c1u0, c2u0, betau0)

# Calculate MAF
MAF_D1 = calculate_MAF(SdT1_0p5_interp, MAF_SdT1_0p5_interp, fragilityD1, frag_lim_D1)
MAF_u0 = calculate_MAF(PGV_interp, MAF_PGV_interp, fragilityu0, frag_lim_u0)

print(f"The MAF(D1>{frag_lim_D1:.3g}m) is {MAF_D1:.2e}")
print(f"The MAF(u0>{frag_lim_u0:.1g}m) is {MAF_u0:.2e}")

The MAF(D1>0.084m) is 1.28e-16
The MAF(u0>0.4m) is 1.07e-03


In [5]:
# Change the friction coefficient to 0.06 and repeat the procedure:
T1 = 0.5 # possible from 0.1 to 1, in seconds
mratio = 0.55 # possible from 0.3 to 0.9
muf = 0.06 # possible from 0.03 to 0.18
Tb = 4.5 # possible from 3 to 6, in seconds

# Calculate PCE
c1D1, c2D1, betaD1 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'D1','GM')
c1u0, c2u0, betau0 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'u0','GM')

# Calculate fragility
fragilityD1 = frag_from_coefs(frag_lim_D1, SdT1_0p5_interp, c1D1, c2D1, betaD1)
fragilityu0 = frag_from_coefs(frag_lim_u0, PGV_interp, c1u0, c2u0, betau0)

# Calculate MAF
MAF_D1 = calculate_MAF(SdT1_0p5_interp, MAF_SdT1_0p5_interp, fragilityD1, frag_lim_D1)
MAF_u0 = calculate_MAF(PGV_interp, MAF_PGV_interp, fragilityu0, frag_lim_u0)

print(f"The MAF(D1>{frag_lim_D1:.3g}m) is {MAF_D1:.2e}")
print(f"The MAF(u0>{frag_lim_u0:.1g}m) is {MAF_u0:.2e}")

The MAF(D1>0.084m) is 1.11e-16
The MAF(u0>0.4m) is 9.22e-04


In [6]:
# Change the friction coefficient to 0.08 and repeat the procedure:
T1 = 0.5 # possible from 0.1 to 1, in seconds
mratio = 0.55 # possible from 0.3 to 0.9
muf = 0.08 # possible from 0.03 to 0.18
Tb = 4.5 # possible from 3 to 6, in seconds

# Calculate PCE
c1D1, c2D1, betaD1 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'D1','GM')
c1u0, c2u0, betau0 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'u0','GM')

# Calculate fragility
fragilityD1 = frag_from_coefs(frag_lim_D1, SdT1_0p5_interp, c1D1, c2D1, betaD1)
fragilityu0 = frag_from_coefs(frag_lim_u0, PGV_interp, c1u0, c2u0, betau0)

# Calculate MAF
MAF_D1 = calculate_MAF(SdT1_0p5_interp, MAF_SdT1_0p5_interp, fragilityD1, frag_lim_D1)
MAF_u0 = calculate_MAF(PGV_interp, MAF_PGV_interp, fragilityu0, frag_lim_u0)

print(f"The MAF(D1>{frag_lim_D1:.3g}m) is {MAF_D1:.2e}")
print(f"The MAF(u0>{frag_lim_u0:.1g}m) is {MAF_u0:.2e}")

The MAF(D1>0.084m) is 2.13e-15
The MAF(u0>0.4m) is 7.26e-04


In [7]:
# Change the friction coefficient to 0.1 and repeat the procedure:
T1 = 0.5 # possible from 0.1 to 1, in seconds
mratio = 0.55 # possible from 0.3 to 0.9
muf = 0.1 # possible from 0.03 to 0.18
Tb = 4.5 # possible from 3 to 6, in seconds

# Calculate PCE
c1D1, c2D1, betaD1 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'D1','GM')
c1u0, c2u0, betau0 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'u0','GM')

# Calculate fragility
fragilityD1 = frag_from_coefs(frag_lim_D1, SdT1_0p5_interp, c1D1, c2D1, betaD1)
fragilityu0 = frag_from_coefs(frag_lim_u0, PGV_interp, c1u0, c2u0, betau0)

# Calculate MAF
MAF_D1 = calculate_MAF(SdT1_0p5_interp, MAF_SdT1_0p5_interp, fragilityD1, frag_lim_D1)
MAF_u0 = calculate_MAF(PGV_interp, MAF_PGV_interp, fragilityu0, frag_lim_u0)

print(f"The MAF(D1>{frag_lim_D1:.3g}m) is {MAF_D1:.2e}")
print(f"The MAF(u0>{frag_lim_u0:.1g}m) is {MAF_u0:.2e}")

The MAF(D1>0.084m) is 1.31e-13
The MAF(u0>0.4m) is 6.03e-04


In [8]:
# Change the base period to 3.2sec and repeat the procedure:
T1 = 0.5 # possible from 0.1 to 1, in seconds
mratio = 0.55 # possible from 0.3 to 0.9
muf = 0.1 # possible from 0.03 to 0.18
Tb = 3.2 # possible from 3 to 6, in seconds

# Calculate PCE
c1D1, c2D1, betaD1 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'D1','GM')
c1u0, c2u0, betau0 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'u0','GM')

# Calculate fragility
fragilityD1 = frag_from_coefs(frag_lim_D1, SdT1_0p5_interp, c1D1, c2D1, betaD1)
fragilityu0 = frag_from_coefs(frag_lim_u0, PGV_interp, c1u0, c2u0, betau0)

# Calculate MAF
MAF_D1 = calculate_MAF(SdT1_0p5_interp, MAF_SdT1_0p5_interp, fragilityD1, frag_lim_D1)
MAF_u0 = calculate_MAF(PGV_interp, MAF_PGV_interp, fragilityu0, frag_lim_u0)

print(f"The MAF(D1>{frag_lim_D1:.3g}m) is {MAF_D1:.2e}")
print(f"The MAF(u0>{frag_lim_u0:.1g}m) is {MAF_u0:.2e}")

The MAF(D1>0.084m) is 1.87e-11
The MAF(u0>0.4m) is 5.61e-04


In [9]:
# Try for another limit state threshold for u0:
frag_lim_u0 = 0.5 # m

# And the previous input (with Tb = 4.5)
T1 = 0.5 # possible from 0.1 to 1, in seconds
mratio = 0.55 # possible from 0.3 to 0.9
muf = 0.1 # possible from 0.03 to 0.18
Tb = 4.5 # possible from 3 to 6, in seconds

# Calculate PCE
c1D1, c2D1, betaD1 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'D1','GM')
c1u0, c2u0, betau0 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'u0','GM')

# Calculate fragility
fragilityD1 = frag_from_coefs(frag_lim_D1, SdT1_0p5_interp, c1D1, c2D1, betaD1)
fragilityu0 = frag_from_coefs(frag_lim_u0, PGV_interp, c1u0, c2u0, betau0)

# Calculate MAF
MAF_D1 = calculate_MAF(SdT1_0p5_interp, MAF_SdT1_0p5_interp, fragilityD1, frag_lim_D1)
MAF_u0 = calculate_MAF(PGV_interp, MAF_PGV_interp, fragilityu0, frag_lim_u0)

print(f"The MAF(u0>{frag_lim_u0:.1g}m) is {MAF_u0:.2e}")
print(f"The MAF(D1>{frag_lim_D1:.3g}m) is {MAF_D1:.2e}")

The MAF(u0>0.5m) is 3.83e-04
The MAF(D1>0.084m) is 1.31e-13


In [10]:
# Change the superstructure first-mode natural period repeat the procedure:
## Attention! When we change the period we need to change the hazard curve accordingly:
T1 = 0.75 # s

# Calculate PCE
c1D1, c2D1, betaD1 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'D1','GM')
c1u0, c2u0, betau0 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'u0','GM')

# Calculate fragility
fragilityD1 = frag_from_coefs(frag_lim_D1, SdT1_0p75_interp, c1D1, c2D1, betaD1)
fragilityu0 = frag_from_coefs(frag_lim_u0, PGV_interp, c1u0, c2u0, betau0)

# Calculate MAF
MAF_D1 = calculate_MAF(SdT1_0p75_interp, MAF_SdT1_0p75_interp, fragilityD1, frag_lim_D1)
MAF_u0 = calculate_MAF(PGV_interp, MAF_PGV_interp, fragilityu0, frag_lim_u0)

print(f"The MAF(D1>{frag_lim_D1:.3g}m) is {MAF_D1:.2e}")
print(f"The MAF(u0>{frag_lim_u0:.1g}m) is {MAF_u0:.2e}")

The MAF(D1>0.084m) is 4.49e-05
The MAF(u0>0.5m) is 4.16e-04


In [11]:
# Change the period repeat the procedure:
## Attention! When we change the period we need to change the hazard curve accordingly
T1 = 0.6 # s

# Calculate PCE
c1D1, c2D1, betaD1 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'D1','GM')
c1u0, c2u0, betau0 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'u0','GM')

# Calculate fragility
fragilityD1 = frag_from_coefs(frag_lim_D1, SdT1_0p6, c1D1, c2D1, betaD1)
fragilityu0 = frag_from_coefs(frag_lim_u0, PGV_interp, c1u0, c2u0, betau0)

# Calculate MAF
MAF_D1 = calculate_MAF(SdT1_0p6, MAF_SaT1_0p6, fragilityD1, frag_lim_D1)
MAF_u0 = calculate_MAF(PGV_interp, MAF_PGV_interp, fragilityu0, frag_lim_u0)

print(f"The MAF(D1>{frag_lim_D1:.3g}m) is {MAF_D1:.2e}")
print(f"The MAF(u0>{frag_lim_u0:.1g}m) is {MAF_u0:.2e}")

The MAF(D1>0.084m) is 2.06e-08
The MAF(u0>0.5m) is 3.92e-04


In [12]:
# Change the base period repeat the procedure:
Tb = 5 # s

# Calculate PCE
c1D1, c2D1, betaD1 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'D1','GM')
c1u0, c2u0, betau0 = PCECalculator.run_PCE(T1,muf,mratio,Tb,'u0','GM')

# Calculate fragility
fragilityD1 = frag_from_coefs(frag_lim_D1, SdT1_0p6, c1D1, c2D1, betaD1)
fragilityu0 = frag_from_coefs(frag_lim_u0, PGV_interp, c1u0, c2u0, betau0)

# Calculate MAF
MAF_D1 = calculate_MAF(SdT1_0p6, MAF_SaT1_0p6, fragilityD1, frag_lim_D1)
MAF_u0 = calculate_MAF(PGV_interp, MAF_PGV_interp, fragilityu0, frag_lim_u0)

print(f"The MAF(D1>{frag_lim_D1:.3g}m) is {MAF_D1:.2e}")
print(f"The MAF(u0>{frag_lim_u0:.1g}m) is {MAF_u0:.2e}")

The MAF(D1>0.084m) is 1.57e-08
The MAF(u0>0.5m) is 4.06e-04
